In [1]:
import time
#import myutils
import numpy as np
import tensorflow as tf
from keras.layers import Input, Dense, AveragePooling2D, GlobalAveragePooling2D
from keras import backend as K
from keras.models import Model
from keras.applications.inception_resnet_v2 import InceptionResNetV2

Using TensorFlow backend.


# Load CIFAR10 data
Here we use [keras.datasets](https://keras.io/datasets/) which is pretty similar to our <tt>myutils.load_CIFAR10_dataset()</tt> procedure.

In [2]:
n_classes = 10
from keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

n_training = X_train.shape[0]
n_testing = X_test.shape[0]

y_train = y_train.flatten()
y_test  = y_test.flatten()

print( X_train.shape, y_train.shape,  X_test.shape, y_test.shape )

from matplotlib import pyplot as plt
plt.imshow( X_train[0]  )
plt.show()

(50000, 32, 32, 3) (50000,) (10000, 32, 32, 3) (10000,)


<Figure size 640x480 with 1 Axes>

# Create model

In [3]:
from keras.models import Model
from keras.applications.inception_resnet_v2 import InceptionResNetV2

network_names = [ 'incv2' ]

print("Available networks = ", network_names)
cnnid = 0; # int( input("Please choose the CNN network [0-{n}]: ".format(n=len(network_names)-1)) )

selected_network = network_names[cnnid]
print("Selected network: ", selected_network)

input_shape = {
    'incv2'   : (224,224,3)
}[selected_network]

def create_model_incv2():
    tf_input = Input(shape=input_shape)
    base_model = InceptionResNetV2(input_tensor=tf_input, weights='imagenet', include_top=False)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(n_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return base_model, model

create_model = {
    'incv2'    : create_model_incv2
}[selected_network]

Available networks =  ['incv2']
Selected network:  incv2


# Data generator for tensorflow

The feature extraction can process the batches of data. It is common in feeding neural networks in tensorflow.

In [4]:
# tensorflow placeholder for batch of images from CIFAR10 dataset
batch_of_images_placeholder = tf.placeholder("uint8", (None, 32, 32, 3))

batch_size = 64

tf_resize_op = tf.image.resize_images(batch_of_images_placeholder, (224,224), method=0)

In [5]:
print( X_train.shape, y_train.shape,  X_test.shape, y_test.shape )

(50000, 32, 32, 3) (50000,) (10000, 32, 32, 3) (10000,)


In [6]:
# data generator for tensorflow session
from keras.applications.inception_resnet_v2 import preprocess_input as incv2_preprocess_input

preprocess_input =  incv2_preprocess_input

def data_generator(sess,data,labels):
    def generator():
        start = 0
        end = start + batch_size
        n = data.shape[0]
        while True:
            batch_of_images_resized = sess.run(tf_resize_op, {batch_of_images_placeholder: data[start:end]})
            batch_of_images__preprocessed = preprocess_input(batch_of_images_resized)
            batch_of_labels = labels[start:end]
            
            start += batch_size
            end   += batch_size
            if start >= n:
                start = 0
                end = batch_size
            yield (batch_of_images__preprocessed, batch_of_labels)
    return generator

# Create model

In [7]:
sess = tf.InteractiveSession()

In [8]:
K.set_session(sess)
K.set_learning_phase(1)  # 0 - test,  1 - train

In [9]:
base_model, model = create_model()

In [10]:
# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
    print(i, layer.name)

0 input_1
1 conv2d_1
2 batch_normalization_1
3 activation_1
4 conv2d_2
5 batch_normalization_2
6 activation_2
7 conv2d_3
8 batch_normalization_3
9 activation_3
10 max_pooling2d_1
11 conv2d_4
12 batch_normalization_4
13 activation_4
14 conv2d_5
15 batch_normalization_5
16 activation_5
17 max_pooling2d_2
18 conv2d_9
19 batch_normalization_9
20 activation_9
21 conv2d_7
22 conv2d_10
23 batch_normalization_7
24 batch_normalization_10
25 activation_7
26 activation_10
27 average_pooling2d_1
28 conv2d_6
29 conv2d_8
30 conv2d_11
31 conv2d_12
32 batch_normalization_6
33 batch_normalization_8
34 batch_normalization_11
35 batch_normalization_12
36 activation_6
37 activation_8
38 activation_11
39 activation_12
40 mixed_5b
41 conv2d_16
42 batch_normalization_16
43 activation_16
44 conv2d_14
45 conv2d_17
46 batch_normalization_14
47 batch_normalization_17
48 activation_14
49 activation_17
50 conv2d_13
51 conv2d_15
52 conv2d_18
53 batch_normalization_13
54 batch_normalization_15
55 batch_normalization

In [11]:
for layer in base_model.layers:
    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x000001BC119BB080> False
<keras.layers.convolutional.Conv2D object at 0x000001BC118009B0> True
<keras.layers.normalization.BatchNormalization object at 0x000001BC119BB0F0> True
<keras.layers.core.Activation object at 0x000001BC0C8600F0> True
<keras.layers.convolutional.Conv2D object at 0x000001BC1B632E80> True
<keras.layers.normalization.BatchNormalization object at 0x000001BC4698DF98> True
<keras.layers.core.Activation object at 0x000001BC469B1B70> True
<keras.layers.convolutional.Conv2D object at 0x000001BC48A18C88> True
<keras.layers.normalization.BatchNormalization object at 0x000001BC48A8FFD0> True
<keras.layers.core.Activation object at 0x000001BC48AE5320> True
<keras.layers.pooling.MaxPooling2D object at 0x000001BC48B25240> True
<keras.layers.convolutional.Conv2D object at 0x000001BC48A70A90> True
<keras.layers.normalization.BatchNormalization object at 0x000001BC48C01208> True
<keras.layers.core.Activation object at 0x000001BC48B6

<keras.layers.normalization.BatchNormalization object at 0x000001BC5253CF98> True
<keras.layers.core.Activation object at 0x000001BC5231FE80> True
<keras.layers.core.Activation object at 0x000001BC52446908> True
<keras.layers.merge.Concatenate object at 0x000001BC5251A550> True
<keras.layers.convolutional.Conv2D object at 0x000001BC5251ABE0> True
<keras.layers.core.Lambda object at 0x000001BC5263DE10> True
<keras.layers.core.Activation object at 0x000001BC526CE630> True
<keras.layers.convolutional.Conv2D object at 0x000001BC527028D0> True
<keras.layers.normalization.BatchNormalization object at 0x000001BC5271EFD0> True
<keras.layers.core.Activation object at 0x000001BC52702D68> True
<keras.layers.convolutional.Conv2D object at 0x000001BC5273DAC8> True
<keras.layers.normalization.BatchNormalization object at 0x000001BC527EEE80> True
<keras.layers.core.Activation object at 0x000001BC52895A20> True
<keras.layers.convolutional.Conv2D object at 0x000001BC526B5240> True
<keras.layers.convolu

In [12]:
#   first: train only the top layers (which were randomly initialized)
#   i.e. freeze all convolutional  layers
for layer in base_model.layers:
    layer.trainable = True

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [13]:
y_train_one_hot = tf.one_hot( y_train, n_classes ).eval()

In [14]:
data_train_gen = data_generator(sess, X_train, y_train_one_hot )

In [15]:
print( data_train_gen )

<function data_generator.<locals>.generator at 0x000001BC59E49B70>


In [16]:
# train the model on the new data for a "few" epochs
model.fit_generator(data_train_gen(), n_training/batch_size, verbose=1)

Epoch 1/1
782/781 [==============================] - 531s 680ms/step - loss: 1.0241


## Validation

In [17]:
#resize images
images_resized = sess.run(tf_resize_op, {batch_of_images_placeholder: X_test})
images = preprocess_input(images_resized)

In [18]:
print( images.shape )


(10000, 224, 224, 3)


In [19]:
result = base_model.predict(images, verbose=2)

In [20]:
y_pred = [ np.argmax( result[i] ) for i in range(n_testing) ]

In [21]:
np.sum( y_pred == y_test ) / n_testing

0.0

In [22]:
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.001, momentum=0.9), loss='categorical_crossentropy')

In [23]:
model.fit_generator(data_train_gen(), n_training/batch_size, epochs=10, verbose=1)

Epoch 1/10
782/781 [==============================] - 517s 661ms/step - loss: 0.3835
Epoch 2/10
782/781 [==============================] - 514s 657ms/step - loss: 0.3141
Epoch 3/10
782/781 [==============================] - 501s 640ms/step - loss: 0.2883
Epoch 4/10
782/781 [==============================] - 500s 639ms/step - loss: 0.2708
Epoch 5/10
782/781 [==============================] - 508s 650ms/step - loss: 0.2574
Epoch 6/10
782/781 [==============================] - 505s 645ms/step - loss: 0.2465
Epoch 7/10
782/781 [==============================] - 502s 642ms/step - loss: 0.2368
Epoch 8/10
782/781 [==============================] - 502s 643ms/step - loss: 0.2283
Epoch 9/10
782/781 [==============================] - 503s 643ms/step - loss: 0.2203
Epoch 10/10
782/781 [==============================] - 501s 641ms/step - loss: 0.2129


In [24]:
result = model.predict(images, verbose=1)
y_pred = [ np.argmax( result[i] ) for i in range(n_testing) ]
np.sum( y_pred == y_test ) / n_testing

10000/10000 [==============================] - 43s 4ms/step


0.886